In [1]:
from dotenv import load_dotenv
from pydantic import BaseModel

from pharia_inference_sdk.core import (
    CompleteInput,
    ControlModel,
    NoOpTracer,
    Pharia1ChatModel,
    Task,
    TaskSpan,
)

load_dotenv()

True

# How to implement a task using an Aleph Alpha model
0. Define the task (see [here](./how_to_define_a_task.ipynb))
1. Decide which model best suits your use case (for a list of Aleph Alpha control models see [here](https://aleph-alpha-intelligence-layer.readthedocs-hosted.com/en/latest/intelligence_layer.core.html#intelligence_layer.core.LuminousControlModel))
2. Create a `Task` subclass
   1. Pass the Model to the constructor 
   2. Implement your domain logic in `do_run()`
      1. Generate a `Prompt`. Examples for generating prompts are `ControlModel.to_instruct_prompt()`, `PromptTemplate.to_rich_prompt()` or `Prompt.from_text()`
         - We recommend to not write custom `PromptTemplate`s. If necessary, look up the documentation of `PromptTemplate`
      2. Run the model with the prompt
      3. Map the prompt output to the task output class
3. Run and test it

### Example

In [2]:
# Step 0 - Define the input and output types for your task


class TellAJokeTaskInput(BaseModel):
    topic: str


class TellAJokeTaskOutput(BaseModel):
    joke: str


# Step 1 - we want a control model but do not care otherwise. Therefore we use the default. For our case, the Chat models also work.
model_to_use = Pharia1ChatModel()


# Step 2
class TellAJokeTask(Task[TellAJokeTaskInput, TellAJokeTaskOutput]):
    PROMPT_TEMPLATE: str = """Tell me a joke about the following topic:"""

    # Step 2.1
    def __init__(self, model: ControlModel | None = None) -> None:
        self._model = model if model else Pharia1ChatModel()

    # Step 2.2
    def do_run(
        self, input: TellAJokeTaskInput, task_span: TaskSpan
    ) -> TellAJokeTaskOutput:
        # Step 2.2.1
        prompt = self._model.to_instruct_prompt(self.PROMPT_TEMPLATE, input.topic)
        completion_input = CompleteInput(prompt=prompt)
        # Step 2.2.2
        completion = self._model.complete(completion_input, task_span)
        return TellAJokeTaskOutput(joke=completion.completions[0].completion)


TellAJokeTask(model=model_to_use).run(
    TellAJokeTaskInput(topic="Software Engineers"), NoOpTracer()
)

TellAJokeTaskOutput(joke=' \n\n Why did the software engineer decide to become a farmer? Because in the software world, they were constantly debugging their code and trying to grow their user base!')